In [ ]:
# In[23]:
#Load certain variables:
values_dir = repo.joinpath('data/values_for_distributions')
rech_farm = load_obj(values_dir,'rech_rng_farm')
rech_precip_sum,rech_precip_wint = load_obj(values_dir,'rech_rng_precip')
ghb_inland_sum,ghb_inland_wint = load_obj(values_dir,'ghb_rng_model_edge')
wel_rng_mpday = load_obj(values_dir,'wel_rng_mday')
wel_rng_m3pday = np.r_[wel_rng_mpday]*np.prod(farm_size)

from scipy.io import savemat,loadmat
import datetime

def run_MC(tot_it,plotyn=False,silent=True):
    #### MAKE NEW/ADD TO OLD EXPT ####
    check_MC_inputParams()
    runlog = []
    #### VARY PARAMS ####
    it = 0
    while it < tot_it:
        ssm_data = {}
        it += 1

        #Make timestamp
        sep = ''
        ts = datetime.datetime.now().strftime('%m'+sep+'%d'+sep+'%H'+sep+'%M'+sep+'%S')

        #head_inland_sum
        dist_inland = 10000 #m
        low,high = np.r_[ghb_inland_sum]/dist_inland*Lx #ghb values taken from NMGWM, apply same gradient to this model
        low,high = np.r_[-1.1,0] #adjusted from ghb values because model didnt converge for larger values
        parname='head_inland_sum'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        head_inland_sum = val

        #head_inland_wint
        low,high = np.r_[ghb_inland_wint]/dist_inland*Lx #ghb values taken from NMGWM, apply same gradient to this model
        low,high = np.r_[0,3] #ghb values taken from NMGWM, apply same gradient to this model
        parname='head_inland_wint'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        head_inland_wint = val

        #set ghb data and create dicts
        chd_data, ssm_data_base, ghb_data, wel_data_base = make_bc_dicts((head_inland_sum,head_inland_wint))
        save_obj(m.MC_file.parent,wel_data_base,'wel_data_base')
        save_obj(m.MC_file.parent,ssm_data_base,'ssm_data_base')

        if heterogenous==0:
        #HOMOGENOUS ONLY
            #hk
            low,high= (-0.5,2)
            parname='hk'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            hk = np.ones((nlay,nrow,ncol))* (10**val)
            
            ##por: porosity
            #      
            low,high = np.r_[.2,.5]
            parname='por'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            por = val

        elif heterogenous in [1,2]:
            #########HETEROGENOUS ONLY ##############
            ##por: porosity
            #
            low,high = np.r_[.2,.5]
            parname='por1'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            por1 = val
            
            ##por: porosity
            #
            low,high = np.r_[.3,.7]
            parname='por2'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            por2 = val
            
            #hk1
            low= -2
            high = 0
            parname='hk1'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            hk1 = 10**val

            #hk2
            low= 0
            high = 2
            parname='hk2'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            hk2 = 10**val

            #lith_prop
            low= 0
            high = 0.5
            parname='lith_prop'
            val = sample_dist(sts.uniform,1,*(low,high-low))
            add_to_paramdict(m.inputParams,parname,val)
            lith_prop = round(val,2)

            if heterogenous==1:
                #vario_type
                parname='vario_type'
                val = int(round(np.random.rand()))
                add_to_paramdict(m.inputParams,parname,val)
                if val==1:
                    vario_type = 'Gaussian'
                elif val==0:
                    vario_type = 'Exponential'
                else:
                    pass

                #corr_len
                low= 250
                high = 1000
                parname='corr_len'
                val = sample_dist(sts.uniform,1,*(low,high-low))
                add_to_paramdict(m.inputParams,parname,val)
                corr_len = val

                #corr_len_zx
                # equal to lz/lx
                low= .01
                high = .1
                parname='corr_len_zx'
                val = sample_dist(sts.uniform,1,*(low,high-low))
                add_to_paramdict(m.inputParams,parname,val)
                corr_len_zx = val

                #corr_len_yx
                # equal to ly/lx
                low= 0.1
                high = 1
                parname='corr_len_yx'
                val = sample_dist(sts.uniform,1,*(low,high-low))
                add_to_paramdict(m.inputParams,parname,val)
                corr_len_yx = val

                #clay_lyr_yn
                parname='clay_lyr_yn'
                val = sample_dist(sts.binom,1,*(1,.5))
                add_to_paramdict(m.inputParams,parname,val)
                clay_lyr_yn = bool(val)

                #Create hk grid
                mu = np.log(hkSand)
                sill = 1
                lcol = int(corr_len/delr)
                llay = int(corr_len*corr_len_zx/np.mean(delv))
                lrow = int(corr_len*corr_len_yx/delc)
                fft_grid = np.exp(simulationFFT.simulFFT(nrow, nlay, ncol, mu, sill, vario_type, lrow , llay, lcol))
                grid = np.log10(fft_grid)
                lith_props = [lith_prop,1-lith_prop]
                hk_vals = [hk1,hk2]
                hk = truncate_grf(grid,lith_props,hk_vals,log10trans=False,plotyn=plotyn)
                if clay_lyr_yn:
                    lyr_ind = np.where((henry_botm<-2) & (henry_botm >-15))
                    hk[lyr_ind,:,:int(ncol*3/4)] = hk_vals[0]

            elif heterogenous==2: #MPS
                import sgs_mod
                nodes = 25
                search_ellipse=(2000,500,20,0,0,0) #(max,med,min,az,dip,rake)
                grid_size = (Lz,Ly,Lx)
                grid_cells = dim
                rotind= [1,2,0] #simulation in a Y,X,Z grid
                constrain=1
                seed = np.random.randint(0,10000)
                expgridfile,outgrid,rotind,grid_cells_sgems = sgs_mod.snesim_grid(m.name, Path(model_ws),
                                                                                  grid_size,grid_cells,search_ellipse=search_ellipse,
                                                                                  TIfile=None,TIname=None,marg=1-lith_prop,seed=1,nodes=nodes,
                                                                                  nreals=1,output=False,rmfiles=False,rotind=rotind,constrain=constrain)
                outgrid = sgs_mod.read_sgems_grid(expgridfile,grid_cells,grid_cells_sgems)
                outgrid = outgrid.squeeze()
                hk = np.zeros(dim,dtype=np.float)
                hk[np.where(outgrid==0)]=hk1
#                 hk[np.where(outgrid==1)]=hk2
            
            hk1_ind = np.where(hk==hk.max())
            hk2_ind = np.where(hk==hk.min())
            por = np.ones((nlay,nrow,ncol),dtype=np.float)
            por[hk1_ind] = por1
            por[hk2_ind] = por2
            hk[wel_cells] = hk.max()
            np.save(m.MC_file.parent.joinpath('hk'+ts+'.npy'),hk)

        ####### END OF HETEROGENOUS BLOCK ############

        ##vka: ratio of vk/hk
        #      Uniform (1/20,1)
        low= 1/20
        high = 1
        parname='vka'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        vka = val

        ##al: #longitudinal dispersivity (m)
        #      Uniform [0.1,20] #10 from Walther et al
        low= 0.1
        high = 10
        parname='al'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        al = val

        ##dmcoef: #dispersion coefficient (m2/day)
        #      log-uniform [1e-10,1e-5] #2e-9 from Walther et al
        low,high = np.log10([1e-10,1e-5])
        parname='dmcoef'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        dmcoef = 10**val

        ##sy: #specific yield
        #      
        low,high = np.r_[.1,.4]
        parname='sy'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        sy = val

        ##ss: specific storage
        #      
        low,high = np.log10([5.0e-5,5.0e-3])
        parname='ss'
        val = sample_dist(sts.uniform,1,*(low,high-low))
        add_to_paramdict(m.inputParams,parname,val)
        ss = 10**val


        ##wel
        pressure_size = 25*8*2.59e+6 #area of pressure subarea mi*mi*(m2/mi)
        pump_rate_m3perm2 = 118000*1233.48/(pressure_size) #(af/y)*(af/m3)/(m2)

        low,high = np.log10((1e2,1e5))
        low,high = np.log10((1e2,1e3))
        parname='wel'
        val = sample_dist(sts.uniform,n_wells,*(low,high-low))
        wel_flux = val
        for i in range(n_wells):
            parname_temp = parname+str(i)
            add_to_paramdict(m.inputParams,parname_temp,wel_flux[i])

        #write wel data
        ssm_data_base = load_obj(m.MC_file.parent,'ssm_data_base')
        wel_data_base = load_obj(m.MC_file.parent,'wel_data_base')
        wel_data,ssm_data,_ = add_pumping_wells(wel_data_base,ssm_data_base,n_wells,wel_flux,farm_orig,kper_odd)

        ##rech_precip
        low,high = np.log10([1e-4,5e-1])
        val = sample_dist(sts.uniform,1,*(low,high-low))
        parname='rech_precip'
        add_to_paramdict(m.inputParams,parname,val)
        rech_precip = 10**val/(nrow*ncol)

        #rech_farm
        low,high = [0.05,0.2] #percentage of the well extraction
        val = sample_dist(sts.uniform,1,*(low,high-low))
        parname='rech_farm'
        add_to_paramdict(m.inputParams,parname,val)
        rech_farm = [val*flux/np.prod(farm_size) for flux in wel_flux]

        #Assign recharge data
        rech_farm_mat = np.zeros((nrow,ncol),dtype=np.float32)
        for i in range(nfarms):
            rech_farm_mat[farm_loc_list[i]] = rech_farm[i]

        rech_data = {}
        for i in range(len(perlen)):
            if i in kper_even:
                rech_data[i] = rech_precip
            elif i in kper_odd:
                rech_data[i] = rech_farm_mat

        ##riv_stg
        low,high = (.5,1.5)
        val = sample_dist(sts.uniform,1,*(low,high-low))
        parname='riv_stg'
        add_to_paramdict(m.inputParams,parname,val)
        riv_stg = val

        ##riv_cond
        low,high = np.log10((.001,10))
        val = sample_dist(sts.uniform,1,*(low,high-low))
        parname='riv_cond'
        add_to_paramdict(m.inputParams,parname,val)
        riv_cond = val

        #Write river data--take SSM data from WEL!!
        riv_grad = .0005
        '''
        measurements from Google Earth show 7 ft drop over 8 mi = avg .0001 avg gradient
        Google Earth imagery shows that at least some patches of the river stay wet year-round (Oct 19,2016)
        This most likely means that this is a gaining river, at least during the summer.
        
        '''
        
        riv_data,ssm_data = write_river_data(riv_loc,stage,cond,riv_grad,kper_even,ssm_data)

        #Create instances in flopy
        bas = flopy.modflow.ModflowBas(m, ibound, strt=strt)
        if bc_ocean=='CHD' or bc_inland=='CHD' :
            chd = flopy.modflow.ModflowChd(m, stress_period_data=chd_data)
        if bc_ocean=='GHB' or bc_inland=='GHB'or bc_right_edge=='GHB':
            ghb = flopy.modflow.ModflowGhb(m, stress_period_data=ghb_data)

        rch = flopy.modflow.ModflowRch(m, rech=rech_data)
        wel = flopy.modflow.ModflowWel(m, stress_period_data=wel_data, ipakcb=ipakcb)
        riv = flopy.modflow.ModflowRiv(m, stress_period_data=riv_data)
        # Add LPF package to the MODFLOW model
        lpf = flopy.modflow.ModflowLpf(m, hk=hk, vka=vka, ipakcb=ipakcb,laytyp=1,laywet=1,
                                      ss=ss,sy=sy)

        # Add PCG Package to the MODFLOW model
        pcg = flopy.modflow.ModflowPcg(m, hclose=1.0e-8)

        # Add OC package to the MODFLOW model
        oc = flopy.modflow.ModflowOc(m,
                                     stress_period_data=oc_data,
                                     compact=True)

        #Create the basic MT3DMS model structure
        btn = flopy.mt3d.Mt3dBtn(m,
                                 laycon=lpf.laytyp, htop=henry_top,
                                 dz=dis.thickness.get_value(), prsity=por, icbund=icbund,
                                 sconc=sconc, nprs=1,timprs=timprs)
        adv = flopy.mt3d.Mt3dAdv(m, mixelm=-1)
        dsp = flopy.mt3d.Mt3dDsp(m, al=al, dmcoef=dmcoef)
        gcg = flopy.mt3d.Mt3dGcg(m, iter1=50, mxiter=1, isolve=1, cclose=1e-5)
        ssm = flopy.mt3d.Mt3dSsm(m, stress_period_data=ssm_data)

        #vdf = flopy.seawat.SeawatVdf(m, iwtable=0, densemin=0, densemax=0,denseref=1000., denseslp=0.7143, firstdt=1e-3)
        vdf = flopy.seawat.SeawatVdf(m, mtdnconc=1, mfnadvfd=1, nswtcpl=0, iwtable=1,
                                     densemin=0., densemax=0., denseslp=denseslp, denseref=densefresh)

        #Write input
        m.write_input()

        # Try to delete the output files, to prevent accidental use of older files
        flist = [os.path.join(model_ws,'MT3D.CNF'),
                  os.path.join(model_ws,'MT3D001.MAS'),
                  os.path.join(model_ws, modelname + '.hds'),
                  os.path.join(model_ws, 'MT3D001.UCN'),
                  os.path.join(model_ws, 'MT3D001.UCN'),
                  os.path.join(model_ws, modelname + '.cbc')]
        for f in flist:
            try:
                os.remove(f)
            except:
                print('did not find file to remove: {}'.format(f))


        if plotyn:
            m.plot_hk_ibound(rowslice=rowslice,gridon=gridon)
        #Run model
        print('Running iteration {} of {}...'.format(it,tot_it))
        v = m.run_model(silent=silent, report=True)
        for idx in range(-3, 0):
            print(v[1][idx])
        runlog.append(v[0])
        if v[0]:
            #Record final salinity as .npy, also move full CBC and UCN files to expt folder
            _ = record_salinity(m,ts_hms=ts);
            copy_rename(os.path.join(m.model_ws,'MT3D001.UCN'),m.MC_file.parent.joinpath('conc_'+ts+'.UCN').as_posix())
            #copy_rename(os.path.join(m.model_ws,m.name+'.cbc'),m.MC_file.parent.joinpath('cbc_'+ts_hms+'.cbc').as_posix())
            print('Finished iteration {} of {} successfully...\n\n\n\n'.format(it,tot_it))
        else:
            print('Unsuccessful iteration {} of {}, deleting inputParams from this it...\n\n\n\n'.format(it,tot_it))
            #m.inputParams = rem_last_ind_from_dict(m.inputParams)
    inputParams_success,inputParams_fail = filt_inds_from_dict(m.inputParams,np.arange(len(runlog))[runlog])
    inputParams_all = m.inputParams
    inputParams = {'inputParams_all':inputParams_all,
                   'inputParams_success': inputParams_success,
                   'inputParams_fail': inputParams_fail,
                   'runlog':runlog}
    #Save inputParams immediately to prevent accidental destruction of them
    savemat(m.MC_file.parent.joinpath('inputParams.mat').as_posix(),inputParams)
    np.save(m.MC_file.parent.joinpath('inputParams_all.npy'),m.inputParams)
    np.save(m.MC_file.parent.joinpath('runlog.npy'),runlog)
    save_obj(m.MC_file.parent,m.inputParams,'inputParams_all')
    save_obj(m.MC_file.parent,inputParams_success,'inputParams_success')
    save_obj(m.MC_file.parent,m.dis.get_node_coordinates(),'yxz')
    return m.inputParams,ssm_data,runlog
